In [1]:
# Create folder named experiment#1 in processed data, outputs and analysis

# Get data for flow, tessellations, demographics and geographical boundary of WA

# Split data into training and test - create four files: Flows for train and test, region indices for train and test
# Save these files in processed data folder 
# TODO Create visualization and save to outputs of experiement

# Biased sampling on the training data to create two biased datasets - save in processed data folder for experiment1 

# Run single constrained gravity model on biased training data and original data to create three models 
# Run trained models on unbiased test data and create generated flows in outputs 

# Use generated flows from outputs and test true labels from processed data to create fairness metrics and visualizations
# Save this in outputs - fairness.json and fairness.png (for each model)

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('../preprocessing'))
from train_test_processing import * 
sys.path.append(os.path.abspath('../preprocessing'))
from train_test_split_vis import * 

experiment_id = '1'

washington = load_state_or_county_data('../data/WA/boundary.geojson')
flow_df = pd.read_csv('../data/WA/flow.csv')
population_df = pd.read_csv('../data/WA/features.csv')
washington_census_tracts = load_state_or_county_data('../data/WA/tessellation_wpop.geojson')
grid = create_grid(washington.unary_union, 25, washington.crs)

# Split the data into train and test sets
train_output, test_output = flow_train_test_split(flow_df, population_df, washington.crs, grid, experiment_id = experiment_id)

# vis
# plot_grid_and_census_tracts(grid, washington_census_tracts, train_output, test_output)

train_set_flows, test_set_flows, num_geoid_count = filter_flow_data(flow_df, population_df, train_output, test_output, experiment_id = experiment_id)

print(train_set_flows, test_set_flows, num_geoid_count)


In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('../preprocessing'))
from biased_sampling import *

features_df = pd.read_csv("../data/WA/features.csv")
demographics_df = pd.read_csv("../data/WA/demographics.csv")
train_flows = pd.read_csv("../processed_data/1/train/train_flows.csv")

calculate_biased_flow(features_df, demographics_df, train_flows,demographic_column_name='svi', method=1, order="ascending", sampling=False, experiment_id="1", bias_factor=0.5)


/Users/apoorvasheera/Documents/DSSG/Crowd Flow/crowdflow-fairness/preprocessing/biased_sampling.py:28: FutureWarning: Passing 'suffixes' which cause duplicate columns {'geoid_x'} in the result is deprecated and will raise a MergeError in a future version.
  destination_merge = pd.merge(destination_merge, features[['geoid', 'total_population']], how='left', left_on='geoid_d', right_on='geoid')


Saved adjusted flows to ../processed_data/1/train/svi/1_ascending_biased_flows.csv
